In [23]:
import torch
import torch.nn as nn
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.nn.modules.activation import ReLU

In [24]:
class Discriminator(nn.Module):
  def __init__ (self, channels_img, features_d):
    super(Discriminator,self).__init__()
    self.disc = nn.Sequential(
        #64x64
        nn.Conv2d(channels_img, features_d, kernel_size=4,stride=2, padding = 1),#32x32
        nn.LeakyReLU(0.2),
        self._block(features_d,features_d * 2 , 4 , 2 , 1),#16x16
        self._block(features_d * 2,features_d * 4 , 4 , 2 , 1),#8x8
        self._block(features_d * 4,features_d * 8 , 4 , 2 , 1),#4x4
        nn.Conv2d(features_d*8,1, kernel_size=4,stride=2, padding = 0),#1x1
        nn.Sigmoid(),
    )

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )
  def forward(self,x):
    return self.disc(x)

In [25]:
class Generator(nn.Module):
  def __init__ (self,z_dim,channels_img,features_g):
    super(Generator,self).__init__()
    self.net = nn.Sequential(
        # N x z_dim x 1 x 1
        self._block(z_dim,features_g * 16, 4 , 1 , 0),# N x f_g*16 x 4 x 4
        self._block(features_g * 16,features_g * 8, 4 , 2 , 1),# 8x8
        self._block(features_g * 8,features_g * 4, 4 , 2 , 1),# 16x16
        self._block(features_g * 4,features_g * 2, 4 , 2 , 1),# 32x32
        nn.ConvTranspose2d(features_g*2,channels_img,kernel_size=4,stride = 2, padding = 1,),
        nn.Tanh(),
    )
  def _block(self,in_channels,out_channels,kernel_size,stride,padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
    )
  def forward(self,x):
    return self.net(x)

In [26]:
def initialize_weights(model):
  for m in model.modules():
    if isinstance(m,(nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight,0.0,0.02)

In [27]:
def test():
  N, in_channels, H, W = 8, 3, 64, 64
  z_dim = 100

  x = torch.randn((N, in_channels, H, W))
  disc = Discriminator(in_channels,8)
  assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed" # what assert is?

  gen = Generator(z_dim, in_channels, 8)
  z = torch.randn((N, z_dim, 1, 1))
  assert gen(z).shape == (N, in_channels, H, W), "Generator test failed" # what assert is?
  print("success")

In [28]:
test()

success


In [29]:
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [30]:
# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64

In [31]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)

In [32]:
dataset = datasets.MNIST(root = "dataset/",train = True, transform = transforms, download = True)
loader = DataLoader(dataset,batch_size = BATCH_SIZE,shuffle = True)

In [33]:
gen = Generator(Z_DIM, CHANNELS_IMG,FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG,FEATURES_GEN).to(device)
initialize_weights(gen)
initialize_weights(disc)

In [34]:
opt_gen = optim.Adam(gen.parameters(),lr = LEARNING_RATE,betas=(0.5,0.999))
opt_disc = optim.Adam(disc.parameters(),lr = LEARNING_RATE,betas=(0.5,0.999))
criterion = nn.BCELoss()

In [41]:
fixed_noise = torch.randn(32, Z_DIM, 1, 1).to(device)
writen_real = SummaryWriter(f"logs/real")
writen_fake = SummaryWriter(f"logs/fake")
step = 0

In [36]:
gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

In [ ]:
for epoch in range(NUM_EPOCHS):
  for batch_idx, (real, _) in enumerate(loader):
    real = real.to(device)
    noise = torch.randn((BATCH_SIZE,Z_DIM,1,1)).to(device)
    fake = gen(noise)

    disc_real = disc(real).reshape(-1)
    loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake.detach()).reshape(-1)
    loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    loss_disc = (loss_disc_real + loss_disc_fake) / 2
    disc.zero_grad()
    loss_disc.backward()
    opt_disc.step()

    output = disc(fake).reshape(-1)
    loss_gen = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()

    if batch_idx % 100 == 0:
        print(
            f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)} \
              Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
        )

        with torch.no_grad():
            fake = gen(fixed_noise)
            # take out (up to) 32 examples
            img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
            img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

            writen_real.add_image("Real", img_grid_real, global_step=step)
            writen_fake.add_image("Fake", img_grid_fake, global_step=step)

        step += 1

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/ --port=6010